In [ ]:
!pip install xgboost

In [1]:
from pandas import read_csv, DataFrame, to_datetime
from numpy import radians, sin, cos, arcsin, sqrt, mean
from sklearn import ensemble
import time
from datetime import datetime
import xgboost as xgb
import pymongo
import dns
import pandas as pd

In [ ]:
ATLAS_URL = "" #remove ATLAS URI before committing

In [3]:
client = pymongo.MongoClient(ATLAS_URI)

In [4]:
db = client['ProjectsinProgramming']

In [5]:
yellowTaxiTable = db['YellowTaxi']
yellowTaxiTestTable = db['YellowTaxiTest']


### Train XGBoost for Yellow Taxi

In [6]:
train = pd.DataFrame(list(yellowTaxiTable.find()))
test= pd.DataFrame(list(yellowTaxiTestTable.find()))

In [7]:

train

,_id,key,fare_amount,pickup_datetime,passenger_count,distance
0,608328c055a7bf9dfb58cd9a,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,1,1.03
1,608328c055a7bf9dfb58cd9b,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,1,8.45
2,608328c055a7bf9dfb58cd9c,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,2,1.39
3,608328c055a7bf9dfb58cd9d,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,1,2.80
4,608328c055a7bf9dfb58cd9e,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,1,2.00
...,...,...,...,...,...,...
693065,60832b2355a7bf9dfb6360e3,2010-02-12 12:03:21.0000003,9.7,2010-02-12 12:03:21,1,3.23
693066,60832b2355a7bf9dfb6360e4,2011-07-22 11:46:20.0000002,23.7,2011-07-22 11:46:20,4,12.73
693067,60832b2355a7bf9dfb6360e5,2011-09-07 10:29:16.0000002,4.5,2011-09-07 10:29:16,1,0.32
693068,60832b2355a7bf9dfb6360e6,2009-10-12 12:58:42.0000003,6.5,2009-10-12 12:58:42,1,2.27


In [8]:
test

,_id,key,pickup_datetime,passenger_count,distance
0,6083216d55a7bf9dfb438044,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,1,2.32
1,6083216d55a7bf9dfb438045,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,1,2.43
2,6083216d55a7bf9dfb438046,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,1,0.62
3,6083216d55a7bf9dfb438047,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,1,1.96
4,6083216d55a7bf9dfb438048,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,1,5.39
...,...,...,...,...,...
9909,6083217655a7bf9dfb43a6f9,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51,6,2.13
9910,6083217655a7bf9dfb43a6fa,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51,6,3.27
9911,6083217655a7bf9dfb43a6fb,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15,6,19.19
9912,6083217655a7bf9dfb43a6fc,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19,6,8.35


In [9]:
train['pickup_datetime'] = to_datetime(train['pickup_datetime'])
train['hour_of_day'] = train.pickup_datetime.dt.hour.astype(float)
train['day'] = train.pickup_datetime.dt.day.astype(float)
train['week'] = train.pickup_datetime.dt.week.astype(float)
train['month'] = train.pickup_datetime.dt.month.astype(float)
train['day_of_year'] = train.pickup_datetime.dt.dayofyear.astype(float)
train['week_of_year'] = train.pickup_datetime.dt.weekofyear.astype(float)
train['passenger_count'] = train['passenger_count'].astype(float)
train['distance'] = train['distance'].astype(float)

test['pickup_datetime'] = to_datetime(test['pickup_datetime'])
test['hour_of_day'] = test.pickup_datetime.dt.hour.astype(float)
test['day'] = test.pickup_datetime.dt.day.astype(float)
test['week'] = test.pickup_datetime.dt.week.astype(float)
test['month'] = test.pickup_datetime.dt.month.astype(float)
test['day_of_year'] = test.pickup_datetime.dt.dayofyear.astype(float)
test['week_of_year'] = test.pickup_datetime.dt.weekofyear.astype(float)
test['passenger_count'] = test['passenger_count'].astype(float)
test['distance'] = test['distance'].astype(float)

<ipython-input-9-7e0668cc8b12>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train['week'] = train.pickup_datetime.dt.week.astype(float)
<ipython-input-9-7e0668cc8b12>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train['week_of_year'] = train.pickup_datetime.dt.weekofyear.astype(float)
<ipython-input-9-7e0668cc8b12>:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  test['week'] = test.pickup_datetime.dt.week.astype(float)
<ipython-input-9-7e0668cc8b12>:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  test['week_of_year'] = test.pickup_datetime.dt.weekofyear.astype(float)


In [10]:
train_y = train["fare_amount"]
test_key = test["key"]
train_x = train.drop(["fare_amount", "key", "passenger_count"], axis = 1)
train_x = train_x.drop(['pickup_datetime'], axis = 1)
test = test.drop(['pickup_datetime', 'key', "passenger_count"], axis = 1)

In [41]:
# train_x = train_x.drop(["_id"], axis=1)
# test = test.drop(["_id"], axis=1)
train_y.isnull().sum()

0

In [14]:
train_xgb = xgb.DMatrix(train_x, train_y)
test_xgb = xgb.DMatrix(test)

In [15]:
num_round = 2
param = {'max_depth':12, 'eta':0.2,'min_child-weight':2, 'gamma':2, 'booster':'gbtree', 'three-method':'approx', 'normalize_type':'forest', 'rate_drop':0.3, 'eval_metric':'rmse'}
trainer = xgb.train(param, train_xgb, num_round)
trainer.save_model("yellow_cab_model.json")

# loaded_model
# print(train)

[22:24:06] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child-weight", "normalize_type", "rate_drop", "three-method" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [16]:
loaded_model = xgb.Booster()
loaded_model.load_model("yellow_cab_model.json")
predict = loaded_model.predict(test_xgb, ntree_limit = num_round)

/home/abdullahz/.local/lib/python3.8/site-packages/xgboost/core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [18]:
predict

array([ 3.694891 ,  3.782444 ,  2.1022897, ..., 16.806318 ,  7.6901274,
        2.5814695], dtype=float32)

### Train XGBoost for Uber

In [19]:
others = db['UberLyftStuff']

In [20]:
uber_df = pd.DataFrame(list(others.find({"cab_type":"Uber"})))

In [21]:
uber_df['time_stamp'] = to_datetime(uber_df['time_stamp'])
uber_df['hour_of_day'] = uber_df.time_stamp.dt.hour.astype(float)
uber_df['day'] = uber_df.time_stamp.dt.day.astype(float)
uber_df['week'] = uber_df.time_stamp.dt.week.astype(float)
uber_df['month'] = uber_df.time_stamp.dt.month.astype(float)
uber_df['day_of_year'] = uber_df.time_stamp.dt.dayofyear.astype(float)
uber_df['week_of_year'] = uber_df.time_stamp.dt.weekofyear.astype(float)
uber_df['distance'] =  uber_df['distance'].astype(float)

<ipython-input-21-0fbd052ef4a2>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  uber_df['week'] = uber_df.time_stamp.dt.week.astype(float)
<ipython-input-21-0fbd052ef4a2>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  uber_df['week_of_year'] = uber_df.time_stamp.dt.weekofyear.astype(float)


In [22]:
uber_df

,_id,distance,cab_type,time_stamp,price,surge_multiplier,hour_of_day,day,week,month,day_of_year,week_of_year
0,60845ae79aaa8fb6629c145c,1.11,Uber,2018-11-30 22:13:01,12.0,1,22.0,30.0,48.0,11.0,334.0,48.0
1,60845ae79aaa8fb6629c145d,1.11,Uber,2018-12-13 10:50:11,16.0,1,10.0,13.0,50.0,12.0,347.0,50.0
2,60845ae79aaa8fb6629c145e,1.11,Uber,2018-12-13 19:15:03,7.5,1,19.0,13.0,50.0,12.0,347.0,50.0
3,60845ae79aaa8fb6629c145f,1.11,Uber,2018-12-16 23:55:11,7.5,1,23.0,16.0,50.0,12.0,350.0,50.0
4,60845ae79aaa8fb6629c1460,1.11,Uber,2018-12-14 00:40:07,26.0,1,0.0,14.0,50.0,12.0,348.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
385658,60845d6c9aaa8fb662a6a79a,1.00,Uber,2018-12-01 23:53:05,13.0,1,23.0,1.0,48.0,12.0,335.0,48.0
385659,60845d6c9aaa8fb662a6a79b,1.00,Uber,2018-12-01 23:53:05,9.5,1,23.0,1.0,48.0,12.0,335.0,48.0
385660,60845d6c9aaa8fb662a6a79c,1.00,Uber,2018-12-01 23:53:05,NaN,1,23.0,1.0,48.0,12.0,335.0,48.0
385661,60845d6c9aaa8fb662a6a79d,1.00,Uber,2018-12-01 23:53:05,27.0,1,23.0,1.0,48.0,12.0,335.0,48.0


In [23]:
print(uber_df.time_stamp.nunique())
print(uber_df.distance.nunique())
print(uber_df.price.nunique())

30346
422
127


In [24]:
uber_df_y = uber_df['price']
uber_df_x = uber_df.drop(["_id", "cab_type", "time_stamp", "surge_multiplier"], axis = 1)

In [25]:
uber_df_x = uber_df_x.drop(["price"], axis=1)

In [54]:
uber_df_x

,distance,hour_of_day,day,week,month,day_of_year,week_of_year
0,1.11,22.0,30.0,48.0,11.0,334.0,48.0
1,1.11,10.0,13.0,50.0,12.0,347.0,50.0
2,1.11,19.0,13.0,50.0,12.0,347.0,50.0
3,1.11,23.0,16.0,50.0,12.0,350.0,50.0
4,1.11,0.0,14.0,50.0,12.0,348.0,50.0
...,...,...,...,...,...,...,...
385658,1.00,23.0,1.0,48.0,12.0,335.0,48.0
385659,1.00,23.0,1.0,48.0,12.0,335.0,48.0
385660,1.00,23.0,1.0,48.0,12.0,335.0,48.0
385661,1.00,23.0,1.0,48.0,12.0,335.0,48.0


In [58]:
uber_df_y = uber_df_y.fillna(uber_df_y.mean())
uber_df_y

0         12.000000
1         16.000000
2          7.500000
3          7.500000
4         26.000000
            ...    
385658    13.000000
385659     9.500000
385660    15.795343
385661    27.000000
385662    10.000000
Name: price, Length: 385663, dtype: float64

In [59]:
uber_train_xgb = xgb.DMatrix(uber_df_x, uber_df_y)

In [63]:
num_round = 2
param = {'max_depth':12, 'eta':0.2,'min_child-weight':1, 'gamma':2, 'booster':'gbtree', 'three-method':'approx', 'normalize_type':'forest', 'rate_drop':0.3, 'eval_metric':'rmse'}
uber_trainer = xgb.train(xgb_params, uber_train_xgb, num_round)
uber_trainer.save_model("uber_model.json")

[22:32:44] WARNING: ../src/learner.cc:573: 
Parameters: { "n_estimators", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [64]:
predict = uber_trainer.predict(test_xgb, ntree_limit = num_round)

In [65]:
predict

array([12.221079, 12.80715 ,  9.743331, ..., 12.835542, 13.90937 ,
       10.666513], dtype=float32)

### XGBoost for Lyft

In [67]:
lyft_df = pd.DataFrame(list(others.find({"cab_type":"Lyft"})))

In [68]:
lyft_df['time_stamp'] = to_datetime(uber_df['time_stamp'])
lyft_df['hour_of_day'] = lyft_df.time_stamp.dt.hour.astype(float)
lyft_df['day'] = lyft_df.time_stamp.dt.day.astype(float)
lyft_df['week'] = lyft_df.time_stamp.dt.week.astype(float)
lyft_df['month'] = lyft_df.time_stamp.dt.month.astype(float)
lyft_df['day_of_year'] = lyft_df.time_stamp.dt.dayofyear.astype(float)
lyft_df['week_of_year'] = lyft_df.time_stamp.dt.weekofyear.astype(float)
lyft_df['distance'] =  lyft_df['distance'].astype(float)

<ipython-input-68-72f3565e6c0e>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  lyft_df['week'] = lyft_df.time_stamp.dt.week.astype(float)
<ipython-input-68-72f3565e6c0e>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  lyft_df['week_of_year'] = lyft_df.time_stamp.dt.weekofyear.astype(float)


In [69]:
lyft_df_y = lyft_df['price']
lyft_df_x = lyft_df.drop(["_id", "cab_type", "time_stamp", "surge_multiplier"], axis = 1)

In [70]:
lyft_df_x = lyft_df_x.drop(["price"], axis=1)

In [71]:
lyft_df_y = lyft_df_y.fillna(lyft_df_y.mean())
lyft_df_y

0          5.0
1         11.0
2          7.0
3         26.0
4          9.0
          ... 
307403    16.5
307404    22.5
307405     7.0
307406    10.5
307407    32.5
Name: price, Length: 307408, dtype: float64

In [72]:
lyft_train_xgb = xgb.DMatrix(lyft_df_x, lyft_df_y)

In [74]:
num_round = 2
param = {'max_depth':12, 'eta':0.2,'min_child-weight':1, 'gamma':2, 'booster':'gbtree', 'three-method':'approx', 'normalize_type':'forest', 'rate_drop':0.3, 'eval_metric':'rmse'}
lyft_trainer = xgb.train(param, lyft_train_xgb, num_round)
lyft_trainer.save_model("lyft_model.json")

[22:37:22] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child-weight", "normalize_type", "rate_drop", "three-method" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [75]:
predict = lyft_trainer.predict(test_xgb, ntree_limit = num_round)

/home/abdullahz/.local/lib/python3.8/site-packages/xgboost/core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [76]:
predict

array([ 6.663357 ,  6.740643 ,  4.9359202, ..., 11.570581 , 11.570581 ,
        5.3009195], dtype=float32)

In [98]:
pred = {
    "distance" : [10],
    "hour_of_day":[10.0],
    "day":[27.0],
    "week":[5.0],
    "month":[1.0],
    "day_of_year":[27.0],
    "week_of_year":[5.0],
}
pred_df = pd.DataFrame(pred)
test_df = xgb.DMatrix(pred_df)

In [99]:
predict = lyft_trainer.predict(test_df, ntree_limit = num_round)

/home/abdullahz/.local/lib/python3.8/site-packages/xgboost/core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [101]:
predict[0]

11.570581